In [1]:
import re
import json
from lxml import html
import urllib.request

In [2]:
URL_BASE = 'http://www.mathnet.ru'

URL = '''
    http://www.mathnet.ru/php/
    archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017
    &volume=158&issue=1&option_lang=rus&bookID=1621
'''

In [3]:
ANNOTATION_XPATH_EXPR = '''
    //b[contains(text(), 'Аннотация')]/following-sibling::text()[not(preceding-sibling::b[contains(text(), 'Ключевые')])]
'''

TITLE_XPATH_EXPR = '''
    //td//td//td//a[@class='SLink']
'''

KEYWORDS_XPATH_EXPR = '''
    //b[contains(text(), 'Ключевые')]/following-sibling::i
'''

In [12]:
class Porter:
    PERFECTIVEGROUND = re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp

            rv = Porter.I.sub('', rv, 1)
            
            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word
    stem=staticmethod(stem)

In [13]:
def main():
    main_page = html.fromstring(urllib.request.urlopen(URL).read())

    articles = []
    
    links = list(filter(lambda x: x.text is not None, main_page.xpath(TITLE_XPATH_EXPR)))
    for link in links:
        article_page = html.fromstring(urllib.request.urlopen(URL_BASE + link.get("href")).read())

        annotation = str.join('', article_page.xpath(ANNOTATION_XPATH_EXPR))
        annotation = re.sub('[^a-zA-ZА-Яа-я0-9]+', ' ', annotation)
        
        title = link.text;
        
        keywords = re.split(', ', article_page.xpath(KEYWORDS_XPATH_EXPR)[0].text)

        articles.append({
            'title': {
                'origin': title,
                'porter': Porter.stem(title)
            },
            'link': link.get('href'),
            'annotation': {
                'origin': annotation,
                'porter': Porter.stem(annotation)
            },
            'keywords': keywords
        })

    result = {
        'issue': {
            'URL': URL,
            'articles': articles
        }
    }
        
    print(json.dumps(result, ensure_ascii=False, indent=4, sort_keys=True))

In [14]:
main()

{
    "issue": {
        "URL": "http://www.mathnet.ru/php/archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017&volume=158&issue=1&option_lang=rus&bookID=1621",
        "articles": [
            {
                "annotation": {
                    "origin": " В работе рассмотрен класс многочленов типа Капелли в свободной ассоциативной алгебре где произвольное поле счетное множество обобщающий конструкцию кратных многочленов Капелли Приведены основные свойства введенных многочленов В частности указано их разложение через многочлены того же вида и установлены некоторые соотношения между их идеалами Кроме того установлена связь между двойными многочленами Капелли и квазимногочленами Капелли ",
                    "porter": " в работе рассмотрен класс многочленов типа капелли в свободной ассоциативной алгебре где произвольное поле счетное множество обобщающий конструкцию кратных многочленов капелли приведены основные свойства введенных многочленов в частности указано их 